In [ ]:
!nvcc --version
%pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [5]:
%%cuda
#include <stdio.h>
#include <stdlib.h>

#define N 16
#define BLOCK_SIZE 16

__global__ 
void mergeSortShared(int *d_input, int *d_output, int width) {
    __shared__ int sharedMem[BLOCK_SIZE];

    int tid = threadIdx.x;
    int blockStart = blockIdx.x * blockDim.x;

    if (blockStart + tid < N)
        sharedMem[tid] = d_input[blockStart + tid];
    __syncthreads();

    // Bottom-up merge sort within shared memory
    for (int step = 1; step < BLOCK_SIZE; step *= 2) {
        int start = tid * 2 * step;
        if (start < BLOCK_SIZE) {
            int mid = min(start + step, BLOCK_SIZE);
            int end = min(start + 2 * step, BLOCK_SIZE);

            // Merge two sorted halves: [start..mid-1] and [mid..end-1]
            int i = start, j = mid, k = 0;
            int* temp=(int*)malloc(2 * step * sizeof(int));

            while (i < mid && j < end)
                temp[k++] = (sharedMem[i] < sharedMem[j]) ? sharedMem[i++] : sharedMem[j++];
            while (i < mid)
                temp[k++] = sharedMem[i++];
            while (j < end)
                temp[k++] = sharedMem[j++];

            // Copy back to shared memory
            for (int m = 0; m < (end - start); m++)
                sharedMem[start + m] = temp[m];
        }
        __syncthreads();
    }

    if (blockStart + tid < N)
        d_output[blockStart + tid] = sharedMem[tid];
}

int main() {
    int h_input[N], h_output[N];
    int *d_input, *d_output;

    printf("Unsorted array:\n");
    for (int i = 0; i < N; i++) {
        h_input[i] = rand() % 100;
        printf("%d ", h_input[i]);
    }
    printf("\n");

    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_output, N * sizeof(int));

    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);

    dim3 grid((N + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block(BLOCK_SIZE);

    mergeSortShared<<<grid, block>>>(d_input, d_output, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_output, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nSorted array:\n");
    for (int i = 0; i < N; i++) {
        printf("%d ", h_output[i]);
    }
    printf("\n");

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Unsorted array:
41 67 34 0 69 24 78 58 62 64 5 45 81 27 61 91 

Sorted array:
0 5 24 27 34 41 45 58 61 62 64 67 69 78 81 91 



In [6]:
%%cuda
#include <stdio.h>
#include <stdlib.h>

#define N 32
#define BLOCK_SIZE 16

__global__ 
void blockSort(int *d_in, int *d_out) {
    __shared__ 
    int s_data[BLOCK_SIZE];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;

    if (gid < N)
        s_data[tid] = d_in[gid];
    __syncthreads();

    for (int i = 0; i < BLOCK_SIZE; i++) {
        for (int j = 0; j < BLOCK_SIZE - i - 1; j++) {
            if (s_data[j] > s_data[j + 1]) {
                int temp = s_data[j];
                s_data[j] = s_data[j + 1];
                s_data[j + 1] = temp;
            }
        }
        __syncthreads();
    }

    if (gid < N)
        d_out[gid] = s_data[tid];
}

__global__ 
void globalMerge(int *input, int *output, int width) {
    
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int start1 = 2 * width * blockIdx.x;
    int start2 = start1 + width;
    int end1 = start2;
    int end2 = min(start2 + width, N);
    
    int i = start1, j = start2, k = start1;

    while (i < end1 && j < end2) {
        output[k++] = (input[i] < input[j]) ? input[i++] : input[j++];
    }
    while (i < end1) output[k++] = input[i++];
    while (j < end2) output[k++] = input[j++];
}

int main() {
    int h_input[N], h_output[N];
    for (int i = 0; i < N; i++) {
        h_input[i] = rand() % 100;
    }

    printf("Original array:\n");
    for (int i = 0; i < N; i++) printf("%d ", h_input[i]);
    printf("\n");

    int *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_output, N * sizeof(int));

    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);

    blockSort<<<N / BLOCK_SIZE, BLOCK_SIZE>>>(d_input, d_output);
    cudaDeviceSynchronize();

    int *temp;
    for (int width = BLOCK_SIZE; width < N; width *= 2) {
        int blocks = N / (2 * width);
        globalMerge<<<blocks, 1>>>(d_output, d_input, width);
        cudaDeviceSynchronize();

        temp = d_output;
        d_output = d_input;
        d_input = temp;
    }

    cudaMemcpy(h_output, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nSorted array:\n");
    for (int i = 0; i < N; i++) {
        printf("%d ", h_output[i]);
    }
    printf("\n");

    cudaFree(d_input);
    cudaFree(d_output);
    return 0;
}


Original array:
41 67 34 0 69 24 78 58 62 64 5 45 81 27 61 91 95 42 27 36 91 4 2 53 92 82 21 16 18 95 47 26 

Sorted array:
0 2 4 5 16 18 21 24 26 27 27 34 36 41 42 45 47 53 58 61 62 64 67 69 78 81 82 91 91 92 95 95 

